# Developing an ontology
A possible development process:
1. Analyze the domain and the goal of the ontology
2. Consider to reuse existing ontologies
3. Determine the key concepts of the domain
4. Organize concepts in classes and hierarchies among classes and properties
5. Determine the properties of the classes
6. Add constraints (allowed values) on the properties
7. Create the instances
8. Assign values to the properties for each instance

A guide for ontology development can be found here: https://perso.liris.cnrs.fr/alain.mille/enseignements/Ecole_Centrale/What%20is%20an%20ontology%20and%20why%20we%20need%20it.htm

# Ontology modeling 

Model the domain of geographical entities using OWL. 

1. What are the classes? 
2. What are the object and data type properties ?
3. Create object/data property axioms.
4. Create class axioms. 
    4.1. Create class axioms that involve someValuesFrom and allValuesFrom properties.  
    4.2. Create class axioms that involve (qualified) cardinality restrictions.  
5. Add instance/individuals to the ontology. 
6. Add negative property assertions to the ontology. 



# Object property axioms


Answer the following questions using the ontology (called <b>mythology.owl</b>) given below:

     @prefix ex:  <http://example.org/>.
     @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
     @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>.
     @prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
     ex:CuChulainn ex:knownAs [ ex:name "Cú Chulainn" ; ex:meaning "Culann's Hound"@en ] ;
         ex:father ex:Lugh ;
         ex:describedIn ex:UlsterCycle .
     ex:Setanta ex:knownAs [ ex:name "Sétanta" ] .
     ex:Lugh ex:knownAs [ ex:name "Lugh" ; ex:meaning "Flashing light"@en ] , [ ex:name "Ildanach" ] ;
         ex:onlyChild ex:CuChulainn , ex:Setanta ;
         ex:father ex:Cian ;
         ex:mother ex:Ethniu ;
         ex:describedIn ex:UlsterCycle , ex:MythologicalCycle .
     ex:Cian a ex:FirstBorn ; ex:knownAs [ ex:name "Cian" ; ex:meaning "Enduring one"@en ] ;
         ex:consort ex:Ethniu ;
         ex:sibling ex:Cu , ex:Cethen ;
         ex:describedIn ex:MythologicalCycle .
     ex:ScalBalb a ex:FirstBorn ; ex:knownAs [ ex:name "Scal Balb" ; ex:meaning "Mute champion"@en ] ;
         ex:sibling ex:Cu , ex:Cethen , ex:Cian  .
     ex:Cu ex:knownAs [ ex:name "Cú" ; ex:meaning "Hound" ] ; ex:sibling ex:Cethen , ex:Cian .
     ex:Ethniu ex:knownAs [ ex:name "Ethniu" ] ;
         ex:father ex:Balor ;
         ex:describedIn ex:MythologicalCycle .
     ex:Balor ex:knownAs [ ex:name "Balor" ] ,
         [ ex:name "Balór na Súile Nimhe"@ga ; ex:meaning "Balor of the Evil Eye"@en ] ;
         ex:describedIn ex:MythologicalCycle .
     ex:IrishMythology ex:dividedIn ex:MythologicalCycle , ex:UlsterCycle , ex:FenianCycle , ex:CyclesOfTheKings .
     ex:MythologicalCycle ex:followedBy ex:UlsterCycle .
     ex:FenianCycle ex:precededBy ex:UlsterCycle .
     ex:CyclesOfTheKings ex:precededBy ex:FenianCycle . 


Use the Python OWL-RL library to verify your answers.

### Exercise 1: What can be inferred if we add the following axioms?

       ex:father rdfs:subPropertyOf ex:parent .
       ex:mother rdfs:subPropertyOf ex:parent .

Answer

In [2]:
from rdflib import Graph, URIRef, Literal, Namespace
import owlrl.RDFSClosure
from rdflib.namespace import RDF, OWL, RDFS

g = Graph()
g.parse("mythology.owl", format='turtle')
# rdfs reasoning
#rdfs = owlrl.RDFSClosure.RDFS_Semantics(g, False, False, False)
#rdfs.closure()
#rdfs.flush_stored_triples()

query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ex: <http://example.org/>
SELECT ?s ?o
WHERE
{
?s ex:parent ?o .
}
"""
qresult = g.query(query)
for row in qresult:
    print(row[0])
ex = Namespace("http://example.org/")
g.bind("ex", ex)
g.bind("rdfs", RDFS)

g.add((ex.father, RDFS.subPropertyOf, ex.parent))
g.add((ex.mother, RDFS.subPropertyOf, ex.parent))

# owl reasoning
owl = owlrl.CombinedClosure.RDFS_OWLRL_Semantics(g, False, False, False)
owl.closure()
owl.flush_stored_triples()
qresult = g.query(query)
for row in qresult:
    print(row[0],row[1])

http://example.org/Lugh http://example.org/Ethniu
http://example.org/Ethniu http://example.org/Balor
http://example.org/CuChulainn http://example.org/Lugh
http://example.org/Lugh http://example.org/Cian


### Exercise 2: Add one OWL axiom to infer the following:

       ex:Lugh ex:child ex:CuChulainn .
       ex:Cian ex:child ex:Lugh .
       ex:Etniu ex:child ex:Lugh .  
       ex:Balor ex:child ex:Etniu .

Answer

       ex:parent owl:inverseOf ex:child .

### Exercise 3: Add one OWL axiom to infer the following:

    ex:UlsterCycle ex:precededBy ex:MythologicalCycle .
    ex:UlsterCycle ex:followedBy ex:FenianCycle .
    ex:FenianCycle ex:followedBy ex:CyclesOfTheKings .

Answer: 

    ex:precededBy owl:inverseOf ex:followedBy .

### Exercise 4: Add one OWL axiom to infer the following:

    ex:Ethniu ex:consort ex:Cian .

Answer:

    ex:consort a owl:SymmetricProperty .


### Exercise 5

Add one OWL axiom to state that someone cannot be a sibling of themselves.

Answer:

    ex:sibling a owl:IrreflexiveProperty .


### Exercise 6: Add one OWL axiom to infer the triple:

    ex:Lugh ex:maternalGrandparent ex:Balor .

Answer:

        ex:maternalGrandparent owl:propertyChainAxiom ( ex:mother ex:parent ) .

#  Protégé – Ontology Editor


• Download and install Protégé from https://protege.stanford.edu/ 

• Follow the tutorial in [1] to learn how to use Protégé

[1] Horridge, Matthew, et al. "A practical guide to building owl ontologies using protégé 4 and co- ode tools edition1. 2." The university of Manchester 107 (2009). https://people.cs.vt.edu/~kafura/ComputationalThinking/Class-Notes/Tutorial-Highlighted-Day1.pdf

Also here is a video tutorial:
https://www.youtube.com/watch?v=LQ4iW3PO36E

You can also find an addtional tutorial here http://protegeproject.github.io/protege/getting-started/

### Existing ontologies

• w3c good ontologies https://www.w3.org/wiki/Good_Ontologies

• OLS ontologies https://www.ebi.ac.uk/ols/ontologies

• Protégé ontology library https://protegewiki.stanford.edu/wiki/Protege_Ontology_Library

### Exercise 7: Using WINE ontology



Download the wine (https://www.w3.org/TR/owl-guide/wine.rdf) ontology and use the Pellet reasoner to perform inference. You need to set plug-in for Pellet reasoner.

1- How many axioms are there in the ontology?

2- How many classes, object properties, and data properties are there?